<a href="https://colab.research.google.com/github/PrarabdhDar/Car_Damage_Data_Visualization/blob/main/Facial_Expression_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Unzipping data file
!unzip face-expression-recognition-dataset.zip

Archive:  face-expression-recognition-dataset.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of face-expression-recognition-dataset.zip or
        face-expression-recognition-dataset.zip.zip, and cannot find face-expression-recognition-dataset.zip.ZIP, period.


In [ ]:
#Storing directory locations
train_dir = 'face-expression-recognition-dataset/train'
val_dir = 'face-expression-recognition-dataset/validation'

In [ ]:
#Importing relevant libraries
import os
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D
from keras.models import load_model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [ ]:
#Data Preparation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range = 30,
    shear_range = 0.3,
    zoom_range = 0.3,
    width_shift_range = 0.4,
    height_shift_range = 0.4,
    horizontal_flip = True,
    fill_mode = 'nearest')
val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
batch_size = 8

In [ ]:
#Data Extraction
train_generator = train_datagen.flow_from_directory(
    directory= train_dir,
    target_size=(48, 48),
    color_mode="grayscale",
    batch_size= batch_size,
    class_mode="categorical",
    shuffle=True,
    seed=42
)
val_generator = val_datagen.flow_from_directory(
    directory= val_dir,
    target_size=(48,48),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

FileNotFoundError: ignored

In [ ]:
#Model
model = tf.keras.Sequential()

model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(48, 48, 1)))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(48, 48, 1)))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [ ]:
#Hidden Layer
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [ ]:
#Hidden Layer
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))


In [ ]:
#Hidden Layer
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [ ]:
#Hidden Layer
model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(Dropout(0.5))

In [ ]:
#Hidden Layer
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(Dropout(0.5))

In [ ]:
#Output Layer
model.add(Dense(5, kernel_initializer='he_normal'))
model.add(Activation('softmax'))
print(model.summary())

In [ ]:
checkpoint = ModelCheckpoint('best_weights.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=5,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks = [earlystop, checkpoint,reduce_lr]

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer = Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
train_samples = 24282
val_samples = 5937

In [ ]:
history=model.fit(
                train_generator,
                steps_per_epoch=train_samples//batch_size,
                epochs=42,
                callbacks=callbacks,
                validation_data=val_generator,
                validation_steps=val_samples//batch_size)